In [1]:
import pandas as pd
import numpy as np
import sqldf
import requests
import io
import zipfile
import bs4
import re
import string
import matplotlib.pyplot as plt

# tratamento dados

In [93]:
#exemplo de dados
# BPA = balanço patrimonial ativo
# BPP = balanço patrimonial passivo
# DRE = demonstrativo de resultado

class Get_dados():
    def __init__(self, tipo_dado, ano):
        self.tipo_dado = tipo_dado.upper()
        self.ano = ano.upper()
        
    def pegar_dados(self):

        dados = f'dfp_cia_aberta_{self.tipo_dado}_con_{self.ano}.csv'
        link = f'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_{self.ano}.zip'

        r = requests.get(link)

        file = zipfile.ZipFile(io.BytesIO(r.content))
        file = file.open(dados)
        linhas = file.readlines()
        
        linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
        linhas = [i.split(';') for i in linhas]
        
        df = pd.DataFrame(linhas[1:], columns=linhas[0])
        
        return df

#-------------Ticker ação------------------
def cvm_ticker(ano):

    dados = f'fca_cia_aberta_valor_mobiliario_{ano}.csv'
    link = f'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/fca_cia_aberta_{ano}.zip'

    r = requests.get(link)

    file = zipfile.ZipFile(io.BytesIO(r.content))
    file = file.open(dados)
    linhas = file.readlines()
    
    linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
    linhas = [i.split(';') for i in linhas]
    
    df = pd.DataFrame(linhas[1:], columns=linhas[0])
    
    return df


#---------------------Segmento de atuação-----------------
def cvm_segmento(ano):

    dados = f'fca_cia_aberta_geral_{ano}.csv'
    link = f'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/fca_cia_aberta_{ano}.zip'

    r = requests.get(link)

    file = zipfile.ZipFile(io.BytesIO(r.content))
    file = file.open(dados)
    linhas = file.readlines()
    
    linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
    linhas = [i.split(';') for i in linhas]
    
    df = pd.DataFrame(linhas[1:], columns=linhas[0])
    
    return df


#--------------------Correções---------------------
def corrigir_cnpj(cnpj):
    cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')

    return cnpj
def pass_numeric(serie):
    serie = pd.to_numeric(serie)
    return serie



In [97]:
#=====================================Tratamento balanço patrimonial (ativo) BPA======================================
df_ativo = Get_dados('bpa','2021').pegar_dados()
df_ativo.CNPJ_CIA = corrigir_cnpj(df_ativo.CNPJ_CIA)
df_ativo.VL_CONTA = pass_numeric(df_ativo.VL_CONTA)

#=====================================Tratamento balanço patrimonial (passivo) BPP===================================
df_passivo = Get_dados('bpp','2021').pegar_dados()
df_passivo.CNPJ_CIA = corrigir_cnpj(df_passivo.CNPJ_CIA)
df_passivo.VL_CONTA = pass_numeric(df_passivo.VL_CONTA)

#=====================================Tratamento para o DRE==========================================================
df_dre = Get_dados('dre','2021').pegar_dados()
df_dre.CNPJ_CIA = corrigir_cnpj(df_dre.CNPJ_CIA)
df_dre.VL_CONTA = pass_numeric(df_dre.VL_CONTA)

#====================================Concatenacao ativo+passivo=====================================================
new_df = pd.concat([df_ativo,df_passivo])

#====================================Query para bp==================================================================
query_bp = """

select *
from new_df
group by new_df.CNPJ_CIA, new_df.ORDEM_EXERC, new_df.DS_CONTA
order by new_df.CNPJ_CIA, new_df.CD_CONTA
"""
new_bp = sqldf.run(query_bp)
new_bp = new_bp.loc[new_bp.ORDEM_EXERC == 'ÚLTIMO']

new_bp = new_bp.loc[(new_bp['DENOM_CIA'] != 'BRB BCO DE BRASILIA S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB')&
                   (new_bp['DENOM_CIA'] != 'BANCO INTER S.A.')&
                   (new_bp['DENOM_CIA'] != 'PORTO SEGURO S.A.')&
                   (new_bp['DENOM_CIA'] != 'BRAZILIAN FINANCE E REAL ESTATE S.A.')&
                   (new_bp['DENOM_CIA'] != 'FINANCEIRA ALFA S.A.- CRED FINANC E INVS')&
                   (new_bp['DENOM_CIA'] != 'BCO MERCANTIL DO BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'NU HOLDINGS LTD.')&
                   (new_bp['DENOM_CIA'] != 'BANESTES S.A. - BCO EST ESPIRITO SANTO')&
                   (new_bp['DENOM_CIA'] != 'BCO ABC BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BANCO MODAL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO PAN S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO BRADESCO S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO ALFA DE INVESTIMENTO S.A.')&
                   (new_bp['DENOM_CIA'] != 'ITAU UNIBANCO HOLDING S.A.')&
                   (new_bp['DENOM_CIA'] != 'BANCO BMG S/A')&
                   (new_bp['DENOM_CIA'] != 'BCO PINE S.A')&
                   (new_bp['DENOM_CIA'] != 'BCO DAYCOVAL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BANCO RCI BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO SANTANDER (BRASIL) S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO ESTADO DO RIO GRANDE DO SUL S.A.')
                  ]




#====================================Query para dre=====================================================

query_dre = """
select *
from df_dre
group by df_dre.CNPJ_CIA, df_dre.ORDEM_EXERC, df_dre.DS_CONTA
order by df_dre.CNPJ_CIA ,df_dre.CD_CONTA

"""

new_dre = sqldf.run(query_dre)
new_dre = new_dre.loc[new_dre.ORDEM_EXERC == 'ÚLTIMO']


new_dre = new_dre.loc[(new_dre['DENOM_CIA'] != 'BRB BCO DE BRASILIA S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB')&
                   (new_dre['DENOM_CIA'] != 'BANCO INTER S.A.')&
                   (new_dre['DENOM_CIA'] != 'PORTO SEGURO S.A.')&
                   (new_dre['DENOM_CIA'] != 'BRAZILIAN FINANCE E REAL ESTATE S.A.')&
                   (new_dre['DENOM_CIA'] != 'FINANCEIRA ALFA S.A.- CRED FINANC E INVS')&
                   (new_dre['DENOM_CIA'] != 'BCO MERCANTIL DO BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'NU HOLDINGS LTD.')&
                   (new_dre['DENOM_CIA'] != 'BANESTES S.A. - BCO EST ESPIRITO SANTO')&
                   (new_dre['DENOM_CIA'] != 'BCO ABC BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BANCO MODAL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO PAN S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO BRADESCO S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO ALFA DE INVESTIMENTO S.A.')&
                   (new_dre['DENOM_CIA'] != 'ITAU UNIBANCO HOLDING S.A.')&
                   (new_dre['DENOM_CIA'] != 'BANCO BMG S/A')&
                   (new_dre['DENOM_CIA'] != 'BCO PINE S.A')&
                   (new_dre['DENOM_CIA'] != 'BCO DAYCOVAL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BANCO RCI BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO SANTANDER (BRASIL) S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO ESTADO DO RIO GRANDE DO SUL S.A.')
                  ]



#===================================ÍNDICES=====================================
#=====dicionário
# 3.07 ---> Resultado Antes dos Tributos sobre o Lucro (EBIT)
# 3.01 ---> Receita de Venda de Bens e/ou Serviços
# 2.03 ---> Patrmônio Líquido
# 1 ---> ativo total
# 1.02.03 ---> imobilizado
# 1.01.08.03.01 --> financiamento e empréstimos
# 1.01 ---> ativo circulante
# 2.01 ---> passivo circulante
# 3.11 ---> lucro/prejuizo consolidado do período (rol)
# 2.01.04 ---> empréstimos e financiamentos (DOL)>dívida liquida onerosa
# 3.06.02 ---> despesas financeiras
# 2.03.04.05 ----> lucro_retido

indices_ebit  = list(map(list,np.where(new_dre.CD_CONTA == '3.07')))[0]
indices_receita_total = list(map(list,np.where(new_dre.CD_CONTA == '3.01')))[0]
indices_patrimonio_liquido = list(map(list,np.where(new_bp.CD_CONTA == '2.03')))[0]
indices_ativo_total = list(map(list,np.where(new_bp.CD_CONTA == '1')))[0]
indices_imobilizado = list(map(list,np.where(new_bp.CD_CONTA == '1.02.03')))[0]
indices_ativo_circulante = list(map(list,np.where(new_bp.CD_CONTA == '1.01')))[0]
indices_passivo_circulante = list(map(list,np.where(new_bp.CD_CONTA == '2.01')))[0]
indices_passivo_nao_circulante = list(map(list,np.where(new_bp.CD_CONTA == '2.02')))[0]
indices_emprestimos_financiamento = list(map(list,np.where(new_bp.CD_CONTA == '2.01.04')))[0]
indices_lucro_periodo = list(map(list,np.where(new_dre.CD_CONTA == '3.11')))[0]
indices_caixa = list(map(list,np.where(new_bp.CD_CONTA == '1.01.01')))[0]
indices_lucro_retido = list(map(list,np.where(new_bp.CD_CONTA == '2.03.04.05')))[0]
indices_despesas_financeiras = list(map(list,np.where(new_dre.CD_CONTA == '3.06.02')))[0]
indices_imposto_renda = list(map(list,np.where(new_dre.CD_CONTA == '3.08')))[0]


empresas = []
ebit = []
receita_total = []
pl = []
ativo_total = []
imobilizado = []
ativo_circulante = []
passivo_circulante = []
lucro_periodo = []
emprestimos_financiamento = []
cnpj = []
lucro_retido = []
despesas_financeiras = []
imposto_de_renda = []
passivo_nao_circulante = []
#====================passivo não circulante===================
for i in indices_passivo_nao_circulante:
    passivo_nao_circulante.append(new_bp.iloc[i,13])
#====================importo de renda===================
for i in indices_imposto_renda:
    imposto_de_renda.append(new_dre.iloc[i,14])
#====================despesas financeiras===================
for i in indices_despesas_financeiras:
    despesas_financeiras.append(new_dre.iloc[i,14])
    
#====================lucro retido===================
for i in indices_lucro_retido:
    lucro_retido.append(new_bp.iloc[i,13])


#====================lucro do periodo====================
for i in indices_lucro_periodo:
    lucro_periodo.append(new_dre.iloc[i,14])

#====================indices_emprestimos_financiamento====================
for i in indices_emprestimos_financiamento:
    emprestimos_financiamento.append(new_bp.iloc[i,13])
    
#====================EBIT====================
for i in indices_ebit:
    empresas.append(new_dre.iloc[i,4])
    cnpj.append(new_dre.iloc[i,1])
    ebit.append(new_dre.iloc[i,14])

#======================receita total======================
for i in indices_receita_total:
    receita_total.append(new_dre.iloc[i,14])

#=====================Patrimônio líquido=================
for i in indices_patrimonio_liquido:
    pl.append(new_bp.iloc[i,13])

#=====================Ativo total=================
for i in indices_ativo_total:
    ativo_total.append(new_bp.iloc[i,13])
#=====================Imobilizado=================
for i in indices_imobilizado:
    imobilizado.append(new_bp.iloc[i,13])

#=====================ativo circulante=================
for i in indices_ativo_circulante:
    ativo_circulante.append(new_bp.iloc[i,13])

#=====================passivo circulante=================
for i in indices_passivo_circulante:
    passivo_circulante.append(new_bp.iloc[i,13])

    
    
    
#=============================construção df=========================    

df = pd.DataFrame({"cnpj":cnpj,
                    "empresas":empresas,
                   "ativo_total":ativo_total,
                   "ativo_circulante":ativo_circulante,
                   "passivo_circulante":passivo_circulante,
                   "ebit":ebit,
                   "receita_total":receita_total,
                   "capitais_de_terceiros":emprestimos_financiamento,
                   "patrimonio_liquido":pl,
                    "imobilizado":imobilizado,
                    "lucro_periodo":lucro_periodo,
                    "lucro_retido": lucro_retido,
                    "despesas_financeiras":despesas_financeiras,
                    "imposto_de_renda":imposto_de_renda,
                    "passivo_nao_circulante":passivo_nao_circulante})


#==================indicadores estrutura de capital=====================
df['imobilizacao_do_pl'] = round(df.imobilizado/df.patrimonio_liquido*100,2)
df['composicao_endividamento'] = round(df.passivo_circulante/df.capitais_de_terceiros*100,2)
df['participacao_capitais_de_terceiros'] = round(df.capitais_de_terceiros/df.patrimonio_liquido*100,2)
#=======================================================================================================

#==================indicador de liquidez=========================================
df['liquidez_corrente'] = round(df.ativo_circulante/df.passivo_circulante,2)

#==============================indicador de rentabilidade==================================================
df['rentabilidade_ativo'] = round(df.lucro_periodo/df.ativo_total*100,2)

#==========================================================================================================

df['porte_rol'] = round(np.log(df.lucro_periodo*1000),2)
df['porte_ativo'] = round(np.log(df.ativo_total*1000),2)

#df = df.sort_values(by=['ebit/receita_total'], ascending=False)

query_geset = """

select 
df.empresas,
df.imobilizacao_do_pl,
df.composicao_endividamento,
df.participacao_capitais_de_terceiros,
df.liquidez_corrente,
df.rentabilidade_ativo,
df.porte_rol,
df.porte_ativo

from df

"""
risk = sqldf.run(query_geset)


C:\Users\fcaetano\AppData\Local\Temp\ipykernel_10636\674972127.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')
C:\Users\fcaetano\AppData\Local\Temp\ipykernel_10636\674972127.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')
C:\Users\fcaetano\AppData\Local\Temp\ipykernel_10636\674972127.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cnpj = cnpj.str.replace('.','').st

In [98]:
#==================tratamento estatístico=========================
risk = risk.loc[(risk['imobilizacao_do_pl'] != 'inf')&
                (risk['imobilizacao_do_pl'] != 0)&
                
                   (risk['composicao_endividamento'] != 'inf')&
                   (risk['composicao_endividamento'] != 0)&
                
                   (risk['participacao_capitais_de_terceiros'] != 'inf')&
                   (risk['participacao_capitais_de_terceiros'] != 0)&
                
                   (risk['liquidez_corrente'] != 'inf')&
                   (risk['liquidez_corrente'] != 0)&
                
                   (risk['rentabilidade_ativo'] != 'inf')&
                   (risk['rentabilidade_ativo'] != 0)&
                
                   (risk['porte_rol'] != 'inf')&
                   (risk['porte_rol'] != 0)&
                
                
                   (risk['porte_ativo'] != 'inf')&
                   (risk['porte_ativo'] != 0)]

# risk = risk.set_index(['empresas'])
risk = risk.dropna()

df = df.loc[(df['imobilizacao_do_pl'] != 'inf')&
                (df['imobilizacao_do_pl'] != 0)&
                
                   (df['composicao_endividamento'] != 'inf')&
                   (df['composicao_endividamento'] != 0)&
                
                   (df['participacao_capitais_de_terceiros'] != 'inf')&
                   (df['participacao_capitais_de_terceiros'] != 0)&
                
                   (df['liquidez_corrente'] != 'inf')&
                   (df['liquidez_corrente'] != 0)&
                
                   (df['rentabilidade_ativo'] != 'inf')&
                   (df['rentabilidade_ativo'] != 0)&
                
                   (df['porte_rol'] != 'inf')&
                   (df['porte_rol'] != 0)&
                
                
                   (df['porte_ativo'] != 'inf')&
                   (df['porte_ativo'] != 0)]

df = df.dropna()

#base para aplicação do modelo de altman

query_altman = """

select
df.cnpj,
df.empresas,
df.ativo_circulante,
df.passivo_circulante,
df.ativo_total,
df.lucro_periodo,
df.despesas_financeiras,
df.imposto_de_renda,
df.patrimonio_liquido,
df.receita_total,
df.passivo_nao_circulante
from df
"""

df_altman = sqldf.run(query_altman)

df_altman['x1'] = (df_altman.ativo_circulante - df_altman.passivo_circulante )/ df_altman.ativo_total
df_altman['x3'] = (df_altman.lucro_periodo+df_altman.despesas_financeiras + df_altman.imposto_de_renda) / df_altman.ativo_total
df_altman['x4'] = df_altman.patrimonio_liquido / (df_altman.passivo_circulante + df_altman.passivo_nao_circulante)
df_altman['x5'] = df_altman.receita_total / df_altman.ativo_total

                   

In [99]:
im_pl = sorted(risk.imobilizacao_do_pl)
cmp_end = sorted(risk.composicao_endividamento)
prt_cpt_terc = sorted(risk.participacao_capitais_de_terceiros)
liq_corr = sorted(risk.liquidez_corrente)
rent_ativo = sorted(risk.rentabilidade_ativo)
port_rol = sorted(risk.porte_rol)
port_ativo = sorted(risk.porte_ativo)


percent_im_pl = []
percent_cmp_end = []
percent_prt_cpt_terc = []
percent_liq_corr = []
percent_rent_ativo = []
percent_port_rol = []
percent_port_ativo = []

percents = [10.0, 20.0, 30.0, 40.0, 50.0,  60.0,  70.0, 80.0, 90.0]

for i in percents:
    percent_im_pl.append(np.percentile(im_pl,i))
    percent_cmp_end.append(np.percentile(cmp_end,i))
    percent_prt_cpt_terc.append(np.percentile(prt_cpt_terc,i))
    percent_liq_corr.append(np.percentile(liq_corr,i))
    percent_rent_ativo.append(np.percentile(rent_ativo,i))
    percent_port_rol.append(np.percentile(port_rol,i))
    percent_port_ativo.append(np.percentile(port_ativo,i))
    
def decil():  
    pesos = [1,2,3,4,5,6,7,8,9]
    df_decil= pd.DataFrame({"imbolizacao_pl":sorted(percent_im_pl,reverse=True),
                               "composicao_endividamente":sorted(percent_cmp_end,reverse=True),
                               "participacao_capital_de_terceiros":sorted(percent_prt_cpt_terc,reverse=True),
                               "liquidez_corrente":percent_liq_corr,
                               "rentabilidade_ativo":percent_rent_ativo,
                               "porte_rol":percent_port_rol,
                               "porte_ativo":percent_port_ativo,
                               "pesos":pesos})
    return df_decil

def indicadores():

    df_ind = df_percents = pd.DataFrame({"imbolizacao_pl":im_pl,
                               "composicao_endividamente":cmp_end,
                               "participacao_capital_de_terceiros":prt_cpt_terc,
                               "liquidez_corrente":liq_corr,
                               "rentabilidade_ativo":rent_ativo,
                               "porte_rol":port_rol,
                               "porte_ativo":port_ativo})
    return df_ind

In [100]:
#modelo de altman onde:
# x1 = ativo circulante - passivo circulante/ativo total
# x3 = lucro liquido + despesas financeiras + imposto de renda/ativo total
# x4 = patrimônio líquido/ exigível total
# x5 = vendas/ativo total
# nesse modelo o ponto crítico é 0, ou seja z>0 a empresa é solvente e z<0 a empresa é insolvente

def altman(x1,x3,x4,x5):
    z = -1.84 - 0.51*x1 + 6.32*x3 +0.71*x4 + 0.53*x5
    return z

z_value = []
for i in range(len(df_altman.x1)):
    z_value.append(altman(df_altman.iloc[i,11],df_altman.iloc[i,12],df_altman.iloc[i,13],df_altman.iloc[i,14]))

df_altman['z'] = z_value
df_altman = df_altman.loc[(df_altman.lucro_periodo > 0)&(df_altman.receita_total > 0)]
query_altman = """

select
df_altman.cnpj,
df_altman.empresas,
df_altman.x1,
df_altman.x3,
df_altman.x4,
df_altman.x5,
df_altman.z
from df_altman
"""
new_altman = sqldf.run(query_altman)

In [101]:
new_altman

,cnpj,empresas,x1,x3,x4,x5,z
0,00001180000126,CENTRAIS ELET BRAS S.A. - ELETROBRAS,0.084235,-0.042815,0.682986,0.199764,-1.562754
1,00080671000100,CARAMURU ALIMENTOS S.A.,0.199132,-0.119196,0.300815,1.159086,-1.866981
2,00194724000113,AES BRASIL OPERAÇÕES S.A.,0.095407,0.044514,0.644795,0.213427,-1.036410
3,00242184000104,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",0.483237,-0.012496,0.513050,0.118774,-1.738209
4,00389481000179,LM TRANSPORTES INTERESTADUAIS SERVIÇOS E COMÉR...,-0.111345,-0.052857,0.158101,0.397918,-1.794121
...,...,...,...,...,...,...,...
289,92791243000103,IRANI PAPEL E EMBALAGEM S.A.,0.259635,0.034474,0.680065,0.678913,-0.911869
290,96298013000168,PACAEMBU CONSTRUTORA S.A.,0.712755,-0.007439,1.364602,0.951789,-0.777206
291,96418264021802,LOJAS QUERO QUERO S.A.,0.258705,-0.026293,0.251799,0.755406,-1.558970
292,97191902000194,CONSERVAS ODERICH S.A.,0.271655,0.009406,0.757290,0.793419,-0.960912


In [102]:
#-----------------Tratamento segmento----------------
segmento_cvm = cvm_segmento(2021)

query_segmento = """

select 
segmento_cvm.CNPJ_Companhia,
segmento_cvm.Nome_Empresarial,
segmento_cvm.Situacao_Registro_CVM,
segmento_cvm.Setor_Atividade
from segmento_cvm 
group by segmento_cvm.CNPJ_Companhia, segmento_cvm.Nome_Empresarial

"""

segmento_cvm = sqldf.run(query_segmento)
segmento_cvm.CNPJ_Companhia = corrigir_cnpj(segmento_cvm.CNPJ_Companhia)
segmento_cvm = segmento_cvm.dropna()

#--------------Tratamento ticker-----------

ticker_cvm = cvm_ticker(2021)


query_ticker = """

select
ticker_cvm.CNPJ_Companhia,
ticker_cvm.Valor_Mobiliario,
ticker_cvm.Codigo_Negociacao
from ticker_cvm
group by ticker_cvm.CNPJ_Companhia
"""

ticker_cvm = sqldf.run(query_ticker)
ticker_cvm.CNPJ_Companhia = corrigir_cnpj(ticker_cvm.CNPJ_Companhia)
ticker_cvm = ticker_cvm.dropna()

segmento_ticker = ticker_cvm.merge(segmento_cvm,how='inner',left_on='CNPJ_Companhia',right_on='CNPJ_Companhia')
df_rich = segmento_ticker.merge(new_altman,how="inner",left_on="CNPJ_Companhia",right_on="cnpj")




C:\Users\fcaetano\AppData\Local\Temp\ipykernel_10636\674972127.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')
C:\Users\fcaetano\AppData\Local\Temp\ipykernel_10636\674972127.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')


In [103]:
df_rich_v1 = df_rich.loc[(df_rich.Situacao_Registro_CVM != "Certificado de Recebíveis do Agronegócio")&
                      (df_rich.Valor_Mobiliario != "Certificados de depósito de valores mobiliários")&
                      (df_rich.Valor_Mobiliario != "Debêntures")&
                      (df_rich.Valor_Mobiliario != "Nota Comencial")&
                      (df_rich.Valor_Mobiliario != "Valor Mobiliário Não Registrado")&
                      (df_rich.Valor_Mobiliario != "Valor_Mobiliario")
                      ]

In [105]:
df_ativo

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00000000000191,2021-12-31,2,BCO BRASIL S.A.,001023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2020-12-31,1,Ativo Total,1.693794e+09,S
1,00000000000191,2021-12-31,2,BCO BRASIL S.A.,001023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-12-31,1,Ativo Total,1.899341e+09,S
2,00000000000191,2021-12-31,2,BCO BRASIL S.A.,001023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2020-12-31,1.01,Caixa e Equivalentes de Caixa,1.761895e+08,S
3,00000000000191,2021-12-31,2,BCO BRASIL S.A.,001023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-12-31,1.01,Caixa e Equivalentes de Caixa,5.449468e+07,S
4,00000000000191,2021-12-31,2,BCO BRASIL S.A.,001023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2020-12-31,1.01.01,Caixa,1.678456e+07,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58935,97837181000147,2021-12-31,1,DEXCO S.A.,021091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-12-31,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6.110000e+03,N
58936,97837181000147,2021-12-31,1,DEXCO S.A.,021091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2020-12-31,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,1.684300e+05,N
58937,97837181000147,2021-12-31,1,DEXCO S.A.,021091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-12-31,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,1.684300e+05,N
58938,97837181000147,2021-12-31,1,DEXCO S.A.,021091,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2020-12-31,1.02.04.02.11,Goodwill Viva Decora,0.000000e+00,N
